In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

In [5]:
customers = pd.read_csv('customers.csv')
products  = pd.read_csv('products.csv')
transactions  = pd.read_csv('transactions.csv')

In [6]:
# Preprocessing
customer_transactions = transactions.groupby('CustomerID').agg(
    total_spent=('TotalValue', 'sum'),
    total_quantity=('Quantity', 'sum'),
    unique_products=('ProductID', 'nunique'),
    purchase_frequency=('TransactionID', 'count')
).reset_index()
print(customer_transactions)

    CustomerID  total_spent  total_quantity  unique_products  \
0        C0001      3354.52              12                5   
1        C0002      1862.74              10                4   
2        C0003      2725.38              14                4   
3        C0004      5354.88              23                8   
4        C0005      2034.24               7                3   
..         ...          ...             ...              ...   
194      C0196      4982.88              12                3   
195      C0197      1928.65               9                3   
196      C0198       931.83               3                2   
197      C0199      1979.28               9                4   
198      C0200      4758.60              16                5   

     purchase_frequency  
0                     5  
1                     4  
2                     4  
3                     8  
4                     3  
..                  ...  
194                   4  
195                   3

In [7]:
# Merge region data from Customers.csv
customer_profiles = pd.merge(customers, customer_transactions, on='CustomerID', how='right')
print(customer_profiles)

    CustomerID        CustomerName         Region  SignupDate  total_spent  \
0        C0001    Lawrence Carroll  South America  2022-07-10      3354.52   
1        C0002      Elizabeth Lutz           Asia  2022-02-13      1862.74   
2        C0003      Michael Rivera  South America  2024-03-07      2725.38   
3        C0004  Kathleen Rodriguez  South America  2022-10-09      5354.88   
4        C0005         Laura Weber           Asia  2022-08-15      2034.24   
..         ...                 ...            ...         ...          ...   
194      C0196         Laura Watts         Europe  2022-06-07      4982.88   
195      C0197    Christina Harvey         Europe  2023-03-21      1928.65   
196      C0198         Rebecca Ray         Europe  2022-02-27       931.83   
197      C0199      Andrea Jenkins         Europe  2022-12-03      1979.28   
198      C0200         Kelly Cross           Asia  2023-06-11      4758.60   

     total_quantity  unique_products  purchase_frequency  
0   

In [8]:
# Add product category preferences
transactions_products = pd.merge(transactions, products, on='ProductID', how='right')
category_preferences = transactions_products.groupby(['CustomerID', 'Category']).agg(
    category_spent=('TotalValue', 'sum')
).unstack(fill_value=0)
category_preferences.columns = [f"spent_on_{col[1]}" for col in category_preferences.columns]
customer_profiles = pd.merge(customer_profiles, category_preferences, on='CustomerID', how='left')

In [9]:
# Replace NaN values with 0 for missing transaction data
customer_profiles.fillna(0, inplace=True)

In [10]:
# Normalize features for similarity calculation
scaler = MinMaxScaler()
feature_columns = ['total_spent', 'total_quantity', 'unique_products', 'purchase_frequency'] + \
                  [col for col in customer_profiles.columns if col.startswith('spent_on_')]
normalized_features = scaler.fit_transform(customer_profiles[feature_columns])

In [11]:
# Calculate similarity matrix
similarity_matrix = cosine_similarity(normalized_features)
similarity_df = pd.DataFrame(similarity_matrix, index=customer_profiles['CustomerID'], columns=customer_profiles['CustomerID'])
print(similarity_df)

CustomerID     C0001     C0002     C0003     C0004     C0005     C0006  \
CustomerID                                                               
C0001       1.000000  0.624172  0.901406  0.810380  0.934764  0.561489   
C0002       0.624172  1.000000  0.823142  0.869596  0.744514  0.903466   
C0003       0.901406  0.823142  1.000000  0.938927  0.979639  0.738584   
C0004       0.810380  0.869596  0.938927  1.000000  0.901512  0.847347   
C0005       0.934764  0.744514  0.979639  0.901512  1.000000  0.671492   
...              ...       ...       ...       ...       ...       ...   
C0196       0.499776  0.867350  0.753473  0.816104  0.701462  0.944649   
C0197       0.872709  0.808571  0.993722  0.935524  0.982413  0.724786   
C0198       0.451935  0.817146  0.507530  0.536210  0.449039  0.799542   
C0199       0.769781  0.874320  0.948371  0.950144  0.924735  0.760402   
C0200       0.613589  0.908318  0.734415  0.822926  0.661248  0.978841   

CustomerID     C0007     C0008     C0

In [12]:
# top 3 lookalikes for the first 20 customers
lookalikes = {}
for cust_id in customer_profiles['CustomerID'][:20]:
    similar_customers = similarity_df[cust_id].sort_values(ascending=False).iloc[1:4]
    lookalikes[cust_id] = list(zip(similar_customers.index, similar_customers.values))
print(lookalikes)

{'C0001': [('C0069', 0.9930176434209248), ('C0192', 0.9805349862379563), ('C0072', 0.9710717291508056)], 'C0002': [('C0134', 0.9960288508958192), ('C0106', 0.9897010838783545), ('C0036', 0.9835410248090513)], 'C0003': [('C0197', 0.9937218302691064), ('C0166', 0.9912960114173064), ('C0113', 0.982628694249435)], 'C0004': [('C0075', 0.9966789286400511), ('C0090', 0.9892013493405999), ('C0146', 0.9885595812262312)], 'C0005': [('C0085', 0.9927888519265511), ('C0007', 0.9922071097892098), ('C0163', 0.9913612972515684)], 'C0006': [('C0185', 0.9925986688340163), ('C0135', 0.9840741982528924), ('C0200', 0.9788414155569312)], 'C0007': [('C0163', 0.9976183630109238), ('C0085', 0.9931564407426412), ('C0005', 0.9922071097892098)], 'C0008': [('C0024', 0.9872299349862735), ('C0116', 0.9840172241461421), ('C0055', 0.98056300748389)], 'C0009': [('C0098', 0.9740594865316091), ('C0032', 0.9617038157505213), ('C0194', 0.9606646372567466)], 'C0010': [('C0029', 0.9953194979810709), ('C0111', 0.9934571511523

In [13]:
# Save the output as Lookalike.csv
lookalike_df = pd.DataFrame({
    'CustomerID': lookalikes.keys(),
    'Lookalikes': [str(v) for v in lookalikes.values()]
})
lookalike_df.to_csv('FirstName_LastName_Lookalike.csv', index=False)
lookalike_df.head()

,CustomerID,Lookalikes
0,C0001,"[('C0069', 0.9930176434209248), ('C0192', 0.98..."
1,C0002,"[('C0134', 0.9960288508958192), ('C0106', 0.98..."
2,C0003,"[('C0197', 0.9937218302691064), ('C0166', 0.99..."
3,C0004,"[('C0075', 0.9966789286400511), ('C0090', 0.98..."
4,C0005,"[('C0085', 0.9927888519265511), ('C0007', 0.99..."


In [14]:
print(len(lookalike_df))

20
